**Text Mining for AI - Group1 Project** \
First step is acquiring dataset. \
Second step is processing data, for now : \
- we remove duplicates review 
- lower case everything 
- removes rows where the 'app_name' column is null or an empty string

In [14]:
import pandas as pd
import re

df = pd.read_csv('/Volumes/PotatoExpan/Text_Mining_Project/steam_reviews.csv')
#remove dublicate reviews
df = df.drop_duplicates(subset='review_text', keep='first')
#all in lowercase 
df['review_text'] = df['review_text'].str.lower()
df = df[df['review_text'].notna() & (df['review_text'] != '')]
df = df[df['app_name'].notna() & (df['app_name'] != '')]

df = df.drop(columns=['timestamp_created', 'timestamp_updated', 'comment_count', "steam_china_location"])

#print(df.head(10))

/var/folders/43/3l2r907n45j385wwylpn8rym0000gn/T/ipykernel_1575/887350257.py:4: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Volumes/PotatoExpan/Text_Mining_Project/steam_reviews.csv')


Now we only want to keep the english review comments (we are still preprocessing data)

Saving the data as new csv file. 

In [12]:
#df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/processed_steam_reviews.csv')
# Specify a separator and ensure headers are included
#df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/secondtry.csv', sep=';', index=True, header=True)
df.to_csv('/Volumes/PotatoExpan/Text_Mining_Project/colums_removedagain.csv', sep=';', index=False, header=True, line_terminator='\n')

